In [2]:
print("sys.version :" , sys.version)

sys.version : 3.8.4 (tags/v3.8.4:dfa645a, Jul 13 2020, 16:46:45) [MSC v.1924 64 bit (AMD64)]


In [3]:
import redis   # 导入redis 模块

r = redis.Redis(host='localhost', port=6379, decode_responses=True)  
r.set('name', 'runoob')  # 设置 name 对应的值
print(r['name'])
print(r.get('name'))  # 取出键 name 对应的值
print(type(r.get('name')))  # 查看类型

runoob
runoob
<class 'str'>


In [5]:
print(r['num1'], type(r['num1']) )


-1 <class 'str'>


In [1]:
import redis   # 导入redis 模块

r = redis.Redis(host='redis-18029.c11.us-east-1-2.ec2.cloud.redislabs.com', port=18029, password='314159', decode_responses=True)  
# r.set('name', 'runoob')  # 设置 name 对应的值
# print(r['at'])
print(r.get('at'))  # 取出键 name 对应的值
print(type(r.get('at')))  # 查看类型

hwr
<class 'str'>


In [ ]:
r.set('name', 'runoob')

In [11]:
message2_id = r.xadd('mytopic', {'acctid': 103, 'age': 19} )
print(f"\nmessage2_id: {message2_id}")


message2_id: 1615729464581-0


In [12]:
print(f"stream len:{r.xlen('mytopic')}")

stream len:11


In [13]:
print(r.xread({'mytopic': '0-0'}))

[['mytopic', [('1615712911531-0', {'acctid': '012', 'age': '9'}), ('1615712931455-0', {'acctid': '123', 'age': '10'}), ('1615712950666-0', {'acctid': '234', 'age': '10'}), ('1615712999938-0', {'acctid': '345', 'age': '12'}), ('1615713014278-0', {'acctid': '456', 'age': '13'}), ('1615714287129-0', {'acctid': '678', 'age': '14'}), ('1615714671986-0', {'acctid': '789', 'age': '15'}), ('1615715238047-0', {'acctid': '100', 'age': '16'}), ('1615726569993-0', {'acctid': '101', 'age': '17'}), ('1615729314655-0', {'acctid': '102', 'age': '18'}), ('1615729464581-0', {'acctid': '103', 'age': '19'})]]]


In [15]:
print(r.xread({'mytopic': '$'}, block=0))
print(r.xread({'mytopic': '$'}, block=0))

[['mytopic', [('1615731160023-0', {'acctid': '105', 'age': '21'})]]]
[['mytopic', [('1615731175121-0', {'acctid': '106', 'age': '22'})]]]


In [2]:
import threading

from redis import Redis  # 导入包

# r = Redis()  # 初始化Redis对象实例，这里没有填参数，因此会连接本地的redis: 127.0.0.1:6379
r = Redis(host='redis-18029.c11.us-east-1-2.ec2.cloud.redislabs.com', port=18029, password='314159', decode_responses=True)   # 初始化Redis对象实例，这里没有填参数，因此会连接本地的redis: 127.0.0.1:6379

# name = input("what's your name? ")  # 首先要求输入一个名字，作为待会儿群聊时的身份认证标识
# chat_stream = "my_chat_stream"  # 这是这个群聊的一个标识，相当于一个群的名字


def send_msgs():
    # while True:  # 进入死循环
        user_input = input("what you wanna say? ")  # 首先输出你想说啥？提示用户输入内容
        if user_input:
            r.xadd(chat_stream, {name: user_input})  # 然后发送输入的内容到群聊内容里


def handle_msgs(msgs):
    # msgs结构是：[[b'my_chat_stream', [(b'1586416610013-0', {b'jhon': b'nothing'})]]]
    for msg in msgs:  # 迭代，因此msg是 [b'my_chat_stream', [(b'1586416610013-0', {b'jhon': b'nothing'})]]
        _, msg_list = msg  # 解包，因此 msg_list 是 [(b'1586416610013-0', {b'jhon': b'nothing'})]
        for _, content in msg_list:  # 再次解包并迭代，因此 content是 {b'jhon': b'nothing'}
            for user_name, user_input in content.items():  # 迭代，因此user_name是 b'jhon' 而 user_input 是 b'nothing'
                decoded_acctid = acctid.decode()
                decoded_age = age.decode()
                # if decoded_user_nameage == name:
                #     continue

                print("[{}]: {}".format(decoded_acctid, decoded_age))


def recv_msgs():
    while True:
        msgs = r.xread({'mytopic': "$"}, None, 0)  # 获取从群里读出来的内容
        print(msgs)
        handle_msgs(msgs)  # 因为逻辑不算简单，为了这里看起来简单易懂，我们把处理消息的逻辑放在另外一个函数里

# threading.Thread(target=recv_msgs).start()

if __name__ == "__main__":
    threading.Thread(target=recv_msgs).start()
    # send_msgs()



In [3]:
import threading
from redis import Redis  # 导入包

r = Redis(host='redis-18029.c11.us-east-1-2.ec2.cloud.redislabs.com', port=18029, password='314159', decode_responses=True)   # 初始化redis: 127.0.0.1:6379

def recv_msgs():
    while True:
        msgs = r.xread({'mytopic': "$"}, None, block=0)  # 
        print(msgs)
        # handle_msgs(msgs)  # 

# recv_msgs()

# threading.Thread(target=recv_msgs).start()

class MyThread(threading.Thread):
    def __init__(self, thread_name):
        super(MyThread, self).__init__(name = thread_name)

    # 重写run()方法
    def run(self):
        print("%s正在运行中......" % self.name)
        recv_msgs()

i = 1
MyThread("thread-" + str(i)).setDaemon(True).start()   # 启动线程



thread-1正在运行中......


In [1]:
import threading
import time
from redis import Redis  # 导入包

r = Redis(host='redis-18029.c11.us-east-1-2.ec2.cloud.redislabs.com', port=18029, password='314159', decode_responses=True)   # 初始化redis: 127.0.0.1:6379

def recv_msgs():
    while True:
        msgs = r.xread({'mytopic': "$"}, None, block=0)  # 
        print(msgs)
        # handle_msgs(msgs)  # 



t = threading.Thread(target=recv_msgs)  # 注意传入的参数一定是一个元组!
t.setDaemon(True)
t.start()

time.sleep(30)


[['mytopic', [('1615736052577-0', {'acctid': '117', 'age': '33'})]]]
[['mytopic', [('1615736066992-0', {'acctid': '118', 'age': '34'})]]]


In [2]:
import threading

class MyThread(threading.Thread):
    def __init__(self, thread_name):
        super(MyThread, self).__init__(name = thread_name)

    # 重写run()方法
    def run(self):
        print("%s正在运行中......" % self.name)

for i in range(10):
    MyThread("thread-" + str(i)).start()   # 启动线程

thread-0正在运行中......
thread-1正在运行中......
thread-2正在运行中......
thread-3正在运行中......
thread-4正在运行中......
thread-5正在运行中......
thread-6正在运行中......
thread-7正在运行中......
thread-8正在运行中......
thread-9正在运行中......


In [13]:
import threading
import time

def show(arg):
    # time.sleep(1)
    print("thread " + str(arg) + " running......")
    time.sleep(1)

for i in range(10):
    t = threading.Thread(target=show, args=(i,))  # 注意传入的参数一定是一个元组!
    t.setDaemon(True)
    t.start()

thread 0 running......
thread 1 running......
thread 2 running......
thread 3 running......
thread 4 running......
thread 5 running......
thread 6 running......
thread 7 running......
thread 8 running......
thread 9 running......


In [9]:
import time
import threading

def doWaiting():
    print("开始等待：", time.strftime('%H:%M:%S'))
    time.sleep(3)
    print("结束等待：", time.strftime("%H:%M:%S"))

t = threading.Thread(target=doWaiting)
t.setDaemon(True)   # 把子线程设置为守护进程，必须在start()之前设置!!!
t.start()

time.sleep(1)  # 确保线程已经启动
print("开始工作")
print("结束工作")

开始等待： 23:26:53
开始工作
结束工作
